In [13]:
! pip install -q tensorflow numpy pandas scikit-learn mlxtend dataprep transformers
! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./
! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./

     |████████████████████████████████| 92kB 4.4MB/s 
     |████████████████████████████████| 778kB 13.8MB/s 
     |████████████████████████████████| 8.6MB 12.1MB/s 
     |████████████████████████████████| 798kB 56.9MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 1.4MB 50.3MB/s 
     |████████████████████████████████| 368kB 54.0MB/s 
     |████████████████████████████████| 512kB 51.1MB/s 
     |████████████████████████████████| 5.5MB 49.4MB/s 
     |████████████████████████████████| 2.2MB 56.6MB/s 
     |████████████████████████████████| 3.0MB 55.6MB/s 
     |████████████████████████████████| 1.1MB 54.5MB/s 
     |████████████████████████████████| 890kB 47.4MB/s 
     |████████████████████████████████| 655kB 53.1MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
ERROR: panel 0.9.7 has requirement bokeh>=2.1, but you'll

In [ ]:
! ls

In [1]:
import pandas as pd

df = pd.read_csv('dankmemes_task1_train.csv')
embedding = pd.read_csv('dankmemes_task1_train_embeddings.csv', header=None)

FileNotFoundError: ignored

In [2]:
import tensorflow as tf
from tensorflow.keras import Model

class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(1024, activation=tf.keras.activations.relu)
    self.drop = tf.keras.layers.Dropout(rate=0.5)
    self.dense2 = tf.keras.layers.Dense(512, activation=tf.keras.activations.relu)
    self.dense3 = tf.keras.layers.Dense(128, activation=tf.keras.activations.relu)
    self.dense4 = tf.keras.layers.Dense(32, activation=tf.keras.activations.relu)
    self.dense5 = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid) 

  def call(self, x):
    x = self.dense1(x)
    x = self.drop(x)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    return self.dense5(x)

model = MyModel()

In [3]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.RMSprop()

In [4]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

In [5]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [6]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [7]:
def ds_from_df(df, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Meme')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [8]:
def create_map():
  temp = df[['Visual']].values
  counter = 0
  map = {}
  for i in temp:
    for j in i[0].split():
      if j.strip(',') not in map and j.strip(',') != '0':
        map[j.strip(',')] = counter
        counter += 1
  return map

In [9]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [10]:
def augment(training):
  training = np.repeat(training, 10, axis=0)
  for i in range(0, len(temp), 10):
    for j in range(len(X)):
        if (training[i][:1000] == X[j][:1000]).all():
          for k in range(1, 10):
            training[i+k] = np.concatenate([X[i+k][:1000], training[i+k][1000:]])
          break
  return training

In [11]:
def get_X(temp):
  res = []
  for i in range(0, len(temp), 10):
    res.append(temp[i])
  return np.array(res)

In [12]:
import numpy as np
import pickle

X, y = get_X(pickle.load(open('resnest_embeddings.pkl', 'rb'))), df[['Meme']].values
# X, y = np.array([embedding[1][i].split() for i in range(1600)]).astype(float), df[['Meme']].values

FileNotFoundError: ignored

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

ssc = StandardScaler()
mms = MinMaxScaler()

In [ ]:
import pickle
from datetime import date

temp = mms.fit_transform(np.array([(date(int(i[0].split('-')[0]), int(i[0].split('-')[1]), int(i[0].split('-')[2])) - date(2015, 1, 1)).days for i in df[['Date']].values.tolist()]).reshape(1600, 1))
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = ssc.fit_transform(df[['Engagement']].values)
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = ohe(create_map())
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = pickle.load(open('bert_embeddings.pkl', 'rb'))
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

(1600, 1)
(1600, 1001)
(1600, 1)
(1600, 1002)
(1600, 71)
(1600, 1073)
(1600, 768)
(1600, 1841)


In [ ]:
print(X.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
# X_train = augment(X_train)
# y_train = np.repeat(y_train, 10, axis=0)

ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
ds_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
print(ds_train)
print(ds_test)

In [ ]:
buffer_size = 10000
batch_size = 128
num_epochs = 100

In [ ]:
ds_train = ds_train.shuffle(buffer_size=buffer_size,
                            reshuffle_each_iteration=False)
ds_test = ds_test.batch(batch_size)
ds_train = ds_train.batch(batch_size)

In [ ]:
for epoch in range(110):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for features, labels in ds_train:
    train_step(features, labels)

  for test_features, test_labels in ds_test:
    test_step(test_features, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))
  # if test_accuracy.result() > 0.90 and train_accuracy.result() > 0.90:
    # break

In [ ]:
model.save('models/augmented_backpropogation')
! cp -r models ./drive/My\ Drive/Colab\ Notebooks/